<a href="https://colab.research.google.com/github/ykitaguchi77/Streamlit_Colab_Tutorial/blob/main/section_4/02_webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像認識アプリ
Streamlitを使い、画像を認識するアプリを作りましょう。  
フレームワークにはPyTorch使い、オリジナルのCNNモデルを読み込んで使用します。

## ●ライブラリのインストール
Streamlit、およびアプリの動作の確認に使用する「ngrok」をインストールします。

In [1]:
!pip install streamlit==1.7.0 --quiet
!pip install pyngrok==4.1.1 --quiet

     |████████████████████████████████| 9.9 MB 3.6 MB/s 
     |████████████████████████████████| 78 kB 2.6 MB/s 
     |████████████████████████████████| 182 kB 37.4 MB/s 
     |████████████████████████████████| 4.7 MB 24.6 MB/s 
     |████████████████████████████████| 164 kB 35.5 MB/s 
     |████████████████████████████████| 62 kB 561 kB/s 


Streamlit、およびngrokをインポートしておきます。  
エラーが発生する場合は、「ランタイム」→「ランタイムを再起動」によりランタイムを再起動し、再びコードセルを上から順に実行しましょう。

In [2]:
import streamlit as st
from pyngrok import ngrok

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
2022-12-11 17:21:08.089 INFO    numexpr.utils: NumExpr defaulting to 2 threads.


## ●「モデル」を扱うファイル
画像認識の訓練済みモデルを読み込み、予測を行うコードを「model.py」に書き込みます。  

In [3]:
%%writefile model.py
# 以下を「model.py」に書き込み
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

classes_ja = ["飛行機", "自動車", "鳥", "猫", "鹿", "犬", "カエル", "馬", "船", "トラック"]
classes_en = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
n_class = len(classes_ja)
img_size = 32

# CNNのモデル
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 256)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def predict(img):
    # モデルへの入力
    img = img.convert("RGB")
    img = img.resize((img_size, img_size))
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))  # 平均値を0、標準偏差を1に
                                ])
    img = transform(img)
    x = img.reshape(1, 3, img_size, img_size)

    # 訓練済みモデル
    net = Net()
    net.load_state_dict(torch.load(
        "model_cnn.pth", map_location=torch.device("cpu")
        ))
    
    # 予測
    net.eval()
    y = net(x)

    # 結果を返す
    y_prob = torch.nn.functional.softmax(torch.squeeze(y))  # 確率で表す
    sorted_prob, sorted_indices = torch.sort(y_prob, descending=True)  # 降順にソート
    return [(classes_ja[idx], classes_en[idx], prob.item()) for idx, prob in zip(sorted_indices, sorted_prob)]

Writing model.py


## ●アプリのコード
画像認識アプリのコードを、「app.py」に書き込みます。  
ローカルからアップロード、もしくはWebカメラで撮影した画像ファイルに、何が映っているのかを判定します。  
なお、Webカメラはngrokが発行したURLではセキュリティ上動作しないので、今回は動作を確認できません。

In [4]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from model import predict

st.set_option("deprecation.showfileUploaderEncoding", False)

st.sidebar.title("画像認識アプリ")
st.sidebar.write("オリジナルの画像認識モデルを使って何の画像かを判定します。")

st.sidebar.write("")

img_source = st.sidebar.radio("画像のソースを選択してください。",
                              ("画像をアップロード", "カメラで撮影"))
if img_source == "画像をアップロード":
    img_file = st.sidebar.file_uploader("画像を選択してください。", type=["png", "jpg"])
elif img_source == "カメラで撮影":
    img_file = st.camera_input("カメラで撮影")

if img_file is not None:
    with st.spinner("推定中..."):
        img = Image.open(img_file)
        st.image(img, caption="対象の画像", width=480)
        st.write("")

        # 予測
        results = predict(img)

        # 結果の表示
        st.subheader("判定結果")
        n_top = 3  # 確率が高い順に3位まで返す
        for result in results[:n_top]:
            st.write(str(round(result[2]*100, 2)) + "%の確率で" + result[0] + "です。")

        # 円グラフの表示
        pie_labels = [result[1] for result in results[:n_top]]
        pie_labels.append("others")
        pie_probs = [result[2] for result in results[:n_top]]
        pie_probs.append(sum([result[2] for result in results[n_top:]]))
        fig, ax = plt.subplots()
        wedgeprops={"width":0.3, "edgecolor":"white"}
        textprops = {"fontsize":6}
        ax.pie(pie_probs, labels=pie_labels, counterclock=False, startangle=90,
               textprops=textprops, autopct="%.2f", wedgeprops=wedgeprops)  # 円グラフ
        st.pyplot(fig)

Writing app.py


## ●Authtokenの設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [9]:
!ngrok authtoken YourAuthToken

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


## ●アプリの起動と動作確認
streamlitの`run`コマンドでアプリを起動します。


In [6]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [10]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect(port="8501")  # 接続

INFO:pyngrok.process:ngrok process starting: 215
2022-12-11 17:21:52.320 INFO    pyngrok.process: ngrok process starting: 215
INFO:pyngrok.process:t=2022-12-11T17:21:52+0000 lvl=info msg="no configuration paths supplied"

2022-12-11 17:21:52.340 INFO    pyngrok.process: t=2022-12-11T17:21:52+0000 lvl=info msg="no configuration paths supplied"

INFO:pyngrok.process:t=2022-12-11T17:21:52+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2022-12-11 17:21:52.344 INFO    pyngrok.process: t=2022-12-11T17:21:52+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

INFO:pyngrok.process:t=2022-12-11T17:21:52+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2022-12-11 17:21:52.357 INFO    pyngrok.process: t=2022-12-11T17:21:52+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

INFO:pyngrok.process:t=2022-12-11T17:21:52+0000 lvl=info msg="starting web service" obj=we

urlを表示し、リンク先でアプリが動作することを確認します。

In [11]:
print(url)

http://8269-104-155-236-162.ngrok.io


## ●requirements.txtの作成
Streamlit Cloudのサーバー上でアプリを動かすために、「requirements.txt」を作成する必要があります。  
このファイルでは、必要なライブラリのバージョンを指定します。  


まずは、アプリでimportするライブラリのバージョンを確認します。

In [12]:
import streamlit
import torch
import torchvision
import PIL
import matplotlib

print("streamlit==" + streamlit.__version__)
print("torch==" + torch.__version__)
print("torchvision==" + torchvision.__version__)
print("Pillow==" + PIL.__version__)
print("matplotlib==" + matplotlib.__version__)

streamlit==1.7.0
torch==1.13.0+cu116
torchvision==0.14.0+cu116
Pillow==7.1.2
matplotlib==3.2.2


上記を参考に、各ライブラリの望ましいバージョンを記述しrequirements.txtに保存します。

In [ ]:
with open("requirements.txt", "w") as w:
    w.write("streamlit==1.8.1\n")  # Streamlit Cloud上で動作が確認できたバージョン
    w.write("torch==1.10.0\n")  # Cuda対応は要らないのでcu111は記述しない
    w.write("torchvision==0.11.1\n")  # Cuda対応は要らないのでcu111は記述しない
    w.write("Pillow==7.1.2\n")
    w.write("matplotlib==3.2.2\n")

以下の作成されたファイルをダウンロードして、GitHubのレポジトリにアップしましょう。
* app.py
* model.py
* requirements.txt